In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
os.getcwd()

'C:\\Users\\Jalal\\Box\\GNN Project\\Code files'

In [4]:
#pip install netaddr

In [5]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [6]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Thursday_Packet_Level_Data.pkl')

In [7]:
df1 = wed_data.copy()

In [8]:
del wed_data

In [9]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [10]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499343600.000989000,192.168.10.16,209.58.128.160,39846,443,3529,897,00c1b114eb310023ae9b8abf0800,4500003445b240004006d87ec0a80a10d13a80a0,9ba601bb201715c34e47065b8010015cc3a60000010108...,00,fwd,1.0,8.0
1,1499343600.000991000,192.168.10.16,23.60.139.27,46500,80,3537,432,00c1b114eb310023ae9b8abf0800,45000034574e400040067666c0a80a10173c8b1b,b5a40050562f11268d8528b680100112f4ee0000010108...,00,bwd,1.0,8.0
2,1499343600.001028000,192.168.10.16,23.60.139.27,46498,80,3536,432,00c1b114eb310023ae9b8abf0800,45000034cee640004006fecdc0a80a10173c8b1b,b5a200500271988f98a44e71801001008b460000010108...,00,fwd,1.0,8.0
3,1499343600.001547000,216.58.219.226,192.168.10.25,443,49221,3561,3500,002500a8c46000c1b114eb310800,450005b92a8100003506d6dfd83adbe2c0a80a19,01bbc045d127a8cca9a506308018015d92590000010108...,170301058065e6491e2fc3dc9d004ef8bc528b237ab74f...,fwd,1.0,8.0
4,1499343600.001577000,192.168.10.25,216.58.219.226,49221,443,3561,806,00c1b114eb31002500a8c4600800,450000345e9f400040065d46c0a80a19d83adbe2,c04501bba9a50630d127ae518010ffcfc92d0000010108...,00,bwd,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187244,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,bwd,0.0,0.0
7187245,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,bwd,0.0,0.0
7187246,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,bwd,0.0,0.0
7187247,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,bwd,0.0,0.0


In [11]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [12]:
df1['Class Label'].value_counts()

0.0     7087672
11.0      59754
8.0       30059
9.0        9638
10.0        126
Name: Class Label, dtype: int64

In [13]:
df = df1[df1['Class Label']== 8]

In [14]:
df = df.reset_index(inplace = False)

In [15]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,0,1499343600.000989000,192.168.10.16,209.58.128.160,39846,443,3529,897,00c1b114eb310023ae9b8abf0800,4500003445b240004006d87ec0a80a10d13a80a0,9ba601bb201715c34e47065b8010015cc3a60000010108...,00,fwd,1.0,8.0
1,1,1499343600.000991000,192.168.10.16,23.60.139.27,46500,80,3537,432,00c1b114eb310023ae9b8abf0800,45000034574e400040067666c0a80a10173c8b1b,b5a40050562f11268d8528b680100112f4ee0000010108...,00,bwd,1.0,8.0
2,2,1499343600.001028000,192.168.10.16,23.60.139.27,46498,80,3536,432,00c1b114eb310023ae9b8abf0800,45000034cee640004006fecdc0a80a10173c8b1b,b5a200500271988f98a44e71801001008b460000010108...,00,fwd,1.0,8.0
3,3,1499343600.001547000,216.58.219.226,192.168.10.25,443,49221,3561,3500,002500a8c46000c1b114eb310800,450005b92a8100003506d6dfd83adbe2c0a80a19,01bbc045d127a8cca9a506308018015d92590000010108...,170301058065e6491e2fc3dc9d004ef8bc528b237ab74f...,fwd,1.0,8.0
4,4,1499343600.001577000,192.168.10.25,216.58.219.226,49221,443,3561,806,00c1b114eb31002500a8c4600800,450000345e9f400040065d46c0a80a19d83adbe2,c04501bba9a50630d127ae518010ffcfc92d0000010108...,00,bwd,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30054,30054,1499343606.318130000,13.107.4.50,192.168.10.15,80,49910,1295,2862305561,001e4fd4ca2800c1b114eb310800,45001144301540007606e74a0d6b0432c0a80a0f,0050c2f65d09f523ba19026d50100402ed8a0000,d463a1fd5ef8420f7ce16fa653027199e7fee6ee3b4103...,bwd,1.0,8.0
30055,30055,1499343606.318321000,13.107.4.50,192.168.10.15,80,49910,1295,2862309941,001e4fd4ca2800c1b114eb310800,45000b90301840007606ecfb0d6b0432c0a80a0f,0050c2f65d0a063fba19026d50100402e7d60000,9cf7f02feabb81ffc3f11c7e3ec25bc411743295494b26...,fwd,1.0,8.0
30056,30056,1499343606.318351000,192.168.10.15,13.107.4.50,49910,80,1295,207843,00c1b114eb31001e4fd4ca280800,45000028506440008006ce17c0a80a0f0d6b0432,c2f60050ba19026d5d0a11a750100573df8e0000,000000000000,fwd,1.0,8.0
30057,30057,1499343606.318513000,13.107.4.50,192.168.10.15,80,49910,1295,2862312861,001e4fd4ca2800c1b114eb310800,45000b90301a40007606ecf90d6b0432c0a80a0f,0050c2f65d0a11a7ba19026d50100402e7d60000,68bd5ab33d51c06ecb5c0b61c4f8cd9521c1c0d392d556...,fwd,1.0,8.0


In [16]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [17]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [18]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [19]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [20]:
# 70% of benign data for train, 30% for test benign

In [21]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [22]:
#len(imlist)

In [23]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [24]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [25]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [26]:
#len(streams_lst1)

In [27]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.16,209.58.128.160,39846,443,1499343600.000989000,3529,897,fwd,"[0.0, 0.20392156862745098, 0.27058823529411763..."
1,192.168.10.16,23.60.139.27,46500,80,1499343600.000991000,3537,432,bwd,"[0.0, 0.20392156862745098, 0.3411764705882353,..."
2,192.168.10.16,23.60.139.27,46498,80,1499343600.001028000,3536,432,fwd,"[0.0, 0.20392156862745098, 0.807843137254902, ..."
3,216.58.219.226,192.168.10.25,443,49221,1499343600.001547000,3561,3500,fwd,"[0.0196078431372549, 0.7254901960784313, 0.164..."
4,192.168.10.25,216.58.219.226,49221,443,1499343600.001577000,3561,806,bwd,"[0.0, 0.20392156862745098, 0.3686274509803922,..."
...,...,...,...,...,...,...,...,...,...
30054,13.107.4.50,192.168.10.15,80,49910,1499343606.318130000,1295,2862305561,bwd,"[0.06666666666666667, 0.26666666666666666, 0.1..."
30055,13.107.4.50,192.168.10.15,80,49910,1499343606.318321000,1295,2862309941,fwd,"[0.043137254901960784, 0.5647058823529412, 0.1..."
30056,192.168.10.15,13.107.4.50,49910,80,1499343606.318351000,1295,207843,fwd,"[0.0, 0.1568627450980392, 0.3137254901960784, ..."
30057,13.107.4.50,192.168.10.15,80,49910,1499343606.318513000,1295,2862312861,fwd,"[0.043137254901960784, 0.5647058823529412, 0.1..."


In [28]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Thursday_web_bruteforce_all/'

for PKT in range(2, 16):
    
    file_name = f'thursday_bruteforce_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 222
